## Fixed Data:

In [1]:
pip install fredapi

Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
import requests
import pandas as pd
from fredapi import Fred

API_KEY = '1vGb0FOsg2hEIGrEhpueLCcWd1Y'
FRED_API_KEY = 'b13a905b98b5dfc1f3910d6a0d875ae3'

rootUrl = 'https://api.glassnode.com'

symbolList = ['BTC', 'ETH', 'LTC', 'AAVE', 'ABT', 'AMPL', 'ANT', 'ARMOR', 'BADGER', 'BAL', 'BAND', 'BAT', 'BIX', 'BNT', 'BOND', 'BRD', 'BUSD', 'BZRX', 'CELR', 'CHSB', 'CND', 'COMP', 'CREAM', 'CRO', 'CRV', 'CVC', 'CVP', 'DAI', 'DDX', 'DENT', 'DGX', 'DHT', 'DMG', 'DODO', 'DOUGH', 'DRGN', 'ELF', 'ENG', 'ENJ', 'EURS', 'FET', 'FTT', 'FUN', 'GNO', 'GUSD', 'HEGIC', 'HOT', 'HPT', 'HT', 'HUSD', 'INDEX', 'KCS', 'LAMB', 'LBA', 'LDO', 'LEO', 'LINK', 'LOOM', 'LRC', 'MANA', 'MATIC',
              'MCB', 'MCO', 'MFT', 'MIR', 'MKR', 'MLN', 'MTA', 'MTL', 'MX', 'NDX', 'NEXO', 'NFTX', 'NMR', 'Nsure', 'OCEAN', 'OKB', 'OMG', 'PAY', 'PERP', 'PICKLE', 'PNK', 'PNT', 'POLY', 'POWR', 'PPT', 'QASH', 'QKC', 'QNT', 'RDN', 'REN', 'REP', 'RLC', 'ROOK', 'RPL', 'RSR', 'SAI', 'SAN', 'SNT', 'SNX', 'STAKE', 'STORJ', 'sUSD', 'SUSHI', 'TEL', 'TOP', 'UBT', 'UMA', 'UNI', 'USDC', 'USDK', 'USDP', 'USDT', 'UTK', 'VERI', 'WaBi', 'WAX', 'WBTC', 'WETH', 'wNXM', 'WTC', 'YAM', 'YFI', 'ZRX']

symbolListShort = ['BTC', 'ETH', 'LTC']

endPoints = {
    'Price': '/v1/metrics/market/price_usd_close',
    'MarketCap': '/v1/metrics/market/marketcap_usd',
    'TransactionCount': '/v1/metrics/transactions/count',
    'ActiveAddress': '/v1/metrics/addresses/active_count',
}

In [3]:
fred = Fred(api_key=FRED_API_KEY)
Risk_free_rate = fred.get_series('DGS1MO')
Risk_free_rate = Risk_free_rate.to_frame()
Risk_free_rate = Risk_free_rate.reset_index()
Risk_free_rate = Risk_free_rate.rename(columns={'index':'Date', 0:'RiskFree'})

## Get Multiple Attributes for Multiple Coins:

In [4]:
### Define a function to pull data according to input values(coin, feature)
### After running this method, it should return pulled data, and also store it in memory named as "coin_feature"

def PullData(coin, feature):
    print(f"Pulling {coin}'s {feature} data")

    # Assemble full request url and get responses:
    reqUrl = rootUrl + endPoints[feature]
    res = requests.get(reqUrl,
                       params={'a': coin, 'api_key': API_KEY, 'i':'1w'})
    data_raw = pd.read_json(res.text, convert_dates=['t'])

    # Rename column "v" according to features
    data = data_raw.rename(columns={"v" : f"{feature}", 't':'Date'})

    # Turn renamed dataframe into a global variable for future using
    name = f"{coin}_{feature}"
    globals()[name] = data

    # Return a dataframe
    return data

In [27]:
### Get all features(listed in endPoints) of all coins(listed in symbolListShort) 

from functools import reduce

for i in symbolList:
    print(f"Start Pulling {i}'s data:")

    # Preparation for merging
    frameName = f"{i}_Frames"
    locals()[frameName] = []

    # Get data
    for j in list(endPoints.keys()):
        locals()[frameName].append(PullData(i, j))

    # Merge to single dataframe
    print('Merging Dataframes...')
    dfName = f"{i}_All"
    # change merge method to outer to ensure not missing any values.
    locals()[dfName] = reduce(lambda  left,right: pd.merge(left,right, on='Date', how='outer'), locals()[frameName]) 
    
    # change merege method to left, to ensure all the records in api stored.
    locals()[dfName] = locals()[dfName].merge(Risk_free_rate,how = 'left').sort_values(by=['Date'])

    print(f"{i} data successfully pulled\n")
print('\nALL DATA SUCCESSFULLY PULLED!')

Start Pulling BTC's data:
Pulling BTC's Price data
Pulling BTC's MarketCap data
Pulling BTC's TransactionCount data
Pulling BTC's ActiveAddress data
Merging Dataframes...
BTC data successfully pulled

Start Pulling ETH's data:
Pulling ETH's Price data
Pulling ETH's MarketCap data
Pulling ETH's TransactionCount data
Pulling ETH's ActiveAddress data
Merging Dataframes...
ETH data successfully pulled

Start Pulling LTC's data:
Pulling LTC's Price data
Pulling LTC's MarketCap data
Pulling LTC's TransactionCount data
Pulling LTC's ActiveAddress data
Merging Dataframes...
LTC data successfully pulled

Start Pulling AAVE's data:
Pulling AAVE's Price data
Pulling AAVE's MarketCap data
Pulling AAVE's TransactionCount data
Pulling AAVE's ActiveAddress data
Merging Dataframes...
AAVE data successfully pulled

Start Pulling ABT's data:
Pulling ABT's Price data
Pulling ABT's MarketCap data
Pulling ABT's TransactionCount data
Pulling ABT's ActiveAddress data
Merging Dataframes...
ABT data successful

Pulling FET's MarketCap data
Pulling FET's TransactionCount data
Pulling FET's ActiveAddress data
Merging Dataframes...
FET data successfully pulled

Start Pulling FTT's data:
Pulling FTT's Price data
Pulling FTT's MarketCap data
Pulling FTT's TransactionCount data
Pulling FTT's ActiveAddress data
Merging Dataframes...
FTT data successfully pulled

Start Pulling FUN's data:
Pulling FUN's Price data
Pulling FUN's MarketCap data
Pulling FUN's TransactionCount data
Pulling FUN's ActiveAddress data
Merging Dataframes...
FUN data successfully pulled

Start Pulling GNO's data:
Pulling GNO's Price data
Pulling GNO's MarketCap data
Pulling GNO's TransactionCount data
Pulling GNO's ActiveAddress data
Merging Dataframes...
GNO data successfully pulled

Start Pulling GUSD's data:
Pulling GUSD's Price data
Pulling GUSD's MarketCap data
Pulling GUSD's TransactionCount data
Pulling GUSD's ActiveAddress data
Merging Dataframes...
GUSD data successfully pulled

Start Pulling HEGIC's data:
Pulling HEGI

Pulling PICKLE's ActiveAddress data
Merging Dataframes...
PICKLE data successfully pulled

Start Pulling PNK's data:
Pulling PNK's Price data
Pulling PNK's MarketCap data
Pulling PNK's TransactionCount data
Pulling PNK's ActiveAddress data
Merging Dataframes...
PNK data successfully pulled

Start Pulling PNT's data:
Pulling PNT's Price data
Pulling PNT's MarketCap data
Pulling PNT's TransactionCount data
Pulling PNT's ActiveAddress data
Merging Dataframes...
PNT data successfully pulled

Start Pulling POLY's data:
Pulling POLY's Price data
Pulling POLY's MarketCap data
Pulling POLY's TransactionCount data
Pulling POLY's ActiveAddress data
Merging Dataframes...
POLY data successfully pulled

Start Pulling POWR's data:
Pulling POWR's Price data
Pulling POWR's MarketCap data
Pulling POWR's TransactionCount data
Pulling POWR's ActiveAddress data
Merging Dataframes...
POWR data successfully pulled

Start Pulling PPT's data:
Pulling PPT's Price data
Pulling PPT's MarketCap data
Pulling PPT's

Merging Dataframes...
WTC data successfully pulled

Start Pulling YAM's data:
Pulling YAM's Price data
Pulling YAM's MarketCap data
Pulling YAM's TransactionCount data
Pulling YAM's ActiveAddress data
Merging Dataframes...
YAM data successfully pulled

Start Pulling YFI's data:
Pulling YFI's Price data
Pulling YFI's MarketCap data
Pulling YFI's TransactionCount data
Pulling YFI's ActiveAddress data
Merging Dataframes...
YFI data successfully pulled

Start Pulling ZRX's data:
Pulling ZRX's Price data
Pulling ZRX's MarketCap data
Pulling ZRX's TransactionCount data
Pulling ZRX's ActiveAddress data
Merging Dataframes...
ZRX data successfully pulled


ALL DATA SUCCESSFULLY PULLED!


In [89]:
df = pd.DataFrame(columns = LTC_All.columns.values)
df['Asset'] = ''

for i in symbolList:
    
 
    dfName = f"{i}_All" 
    test=pd.DataFrame
    df1 = locals()[dfName][locals()[dfName].Price.notnull()].iloc[0]
    
    df1['Asset'] = i
    df=df.append(df1, ignore_index=True)

    

In [90]:
df.to_csv("firstavailable_price.csv")

In [108]:
new = df.sort_values(by=['Date'])

In [109]:
new.tail()

,Date,Price,MarketCap,TransactionCount,ActiveAddress,RiskFree,Asset
8,2020-11-30,9.141797,0.0,0.0,8907,0.08,BADGER
64,2020-11-30,1.459628,0.0,0.0,2953,0.08,MIR
72,2021-01-04,29.312664,0.0,0.0,950,0.09,NFTX
54,2021-01-04,0.996891,0.0,0.0,1766,0.09,LDO
7,2021-01-18,0.621309,0.0,0.0,2413,NaN,ARMOR


In [98]:
df2.shape

(124, 7)

In [99]:
df2.

TypeError: head() missing 1 required positional argument: 'self'

In [39]:
LTC_All[-52*5:]

,Date,Price,MarketCap,TransactionCount,ActiveAddress,RiskFree
207,2017-05-01,30.281552,1.544286e+09,74069,146364,0.67
208,2017-05-08,28.980451,1.481147e+09,81688,171044,0.73
209,2017-05-15,25.739617,1.318149e+09,82894,187116,0.73
210,2017-05-22,24.132587,1.238288e+09,98181,214016,0.70
211,2017-05-29,27.535853,1.415823e+09,86456,182729,NaN
...,...,...,...,...,...,...
462,2022-03-21,129.241438,9.039261e+09,748866,1514571,0.22
463,2022-03-28,128.926292,9.023978e+09,807733,1566202,0.19
464,2022-04-04,111.343007,7.798857e+09,841901,1576883,0.18
465,2022-04-11,109.018201,7.641529e+09,832321,1652475,0.22


In [77]:
LTC_All[-52:].head()

,Date,Price,MarketCap,TransactionCount,ActiveAddress,RiskFree
415,2021-04-26,269.060433,1.817989e+10,777669,1261380,0.02
416,2021-05-03,385.704466,2.608153e+10,882361,1438322,0.02
417,2021-05-10,296.493781,2.006374e+10,884372,1530001,0.02
418,2021-05-17,144.373194,9.776754e+09,788793,1305682,0.00
419,2021-05-24,170.925525,1.158400e+10,719597,1180854,0.01


In [25]:
BTC_All[BTC_All['Price']== BTC_All.Price.loc[~BTC_All.Price.isnull()].iloc[0]]

,Date,Price,MarketCap,TransactionCount,ActiveAddress,RiskFree
0,2010-07-12,0.08584,295941.984,9847.0,13932,0.16


In [78]:
LTC_All.Price.loc[~LTC_All.Price.isnull()]

0        3.149100
1        3.150000
2        2.830000
3        2.348333
4        2.150000
          ...    
462    129.241438
463    128.926292
464    111.343007
465    109.018201
466    104.648982
Name: Price, Length: 467, dtype: float64

In [103]:
test = AAVE_All[AAVE_All.MarketCap.notnull()]

In [102]:
TEST = locals()[dfName][locals()[dfName].MarketCap.notnull()].iloc[0]

693    False
694    False
615    False
616    False
617    False
       ...  
610     True
611     True
612     True
613     True
614     True
Name: MarketCap, Length: 695, dtype: bool

In [107]:
AAVE_All

,Date,Price,MarketCap,TransactionCount,ActiveAddress,RiskFree
0,2020-09-28,52.675035,0.000000e+00,0,3115,0.09
1,2020-10-05,49.660726,1.546553e+08,0,2736,0.09
2,2020-10-12,40.746978,2.697849e+08,0,3790,NaN
3,2020-10-19,37.887376,3.784656e+08,0,4126,0.09
4,2020-10-26,30.656151,3.337865e+08,0,528,0.08
...,...,...,...,...,...,...
77,2022-03-21,168.881208,2.305997e+09,0,3726,0.22
78,2022-03-28,242.838255,3.317588e+09,0,4618,0.19
79,2022-04-04,178.547260,2.442151e+09,0,2933,0.18
80,2022-04-11,167.536166,2.292021e+09,0,2733,0.22
